In [ ]:
#Importing essential libraries and modules
import tensorflow as tf
from tensorflow.keras.layers import GlobalAveragePooling2D,Input,Dense,Concatenate,Dropout,Flatten
from tensorflow.keras.applications import VGG16,ResNet50,DenseNet121

In [ ]:
from google.colab import files
kaggle=files.upload()

Saving kaggle.json to kaggle.json


In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [ ]:
!kaggle datasets download -d salader/dogs-vs-cats
import zipfile
zip_ref = zipfile.ZipFile('/content/dogs-vs-cats.zip', 'r')
zip_ref.extractall('/content')
zip_ref.close()

Dataset URL: https://www.kaggle.com/datasets/salader/dogs-vs-cats
License(s): unknown
 99% 1.05G/1.06G [00:07<00:00, 175MB/s]
100% 1.06G/1.06G [00:07<00:00, 161MB/s]


In [ ]:
Inputs = Input(shape=(224, 224, 3))
vgg16=VGG16(include_top=False,weights="imagenet",input_tensor=Inputs)
resnet=ResNet50(include_top=False,weights="imagenet",input_tensor=Inputs)
densenet=DenseNet121(include_top=False,weights="imagenet",input_tensor=Inputs)
for layer in vgg16.layers:
    layer.name = "vgg_" + layer.name
for layer in resnet.layers:
    layer.name = "resnet_" + layer.name
for layer in densenet.layers:
    layer.name = "densenet_" + layer.name


58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
29084464/29084464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [ ]:
for model in [vgg16,resnet,densenet]:
  model.trainable=False
vgg_out=GlobalAveragePooling2D()(vgg16.output)
resnet_out=GlobalAveragePooling2D()(resnet.output)
densenet_out=GlobalAveragePooling2D()(densenet.output)

In [ ]:
#Functional API
from tensorflow.keras import Model
concat=Concatenate()([vgg_out,resnet_out,densenet_out])
x=concat
x=Dense(128,activation="relu")(x)
output=Dense(1,activation="sigmoid")(x)
ensemble_model=Model(inputs=Inputs,outputs=output)
ensemble_model.compile(optimizer="adam",loss="binary_crossentropy",metrics=["accuracy"])

In [ ]:
# generators
train_ds = tf.keras.utils.image_dataset_from_directory(
    directory = '/content/train',
    labels='inferred',
    label_mode = 'int',
    batch_size=32,
    image_size=(224,224)
)

validation_ds = tf.keras.utils.image_dataset_from_directory(
    directory = '/content/test',
    labels='inferred',
    label_mode = 'int',
    batch_size=32,
    image_size=(224,224)
)

Found 20000 files belonging to 2 classes.
Found 5000 files belonging to 2 classes.


In [ ]:
# Normalize
def process(image,label):
    image = tf.cast(image/255. ,tf.float32)
    return image,label

train_ds = train_ds.map(process)
validation_ds = validation_ds.map(process)

In [ ]:
ensemble_model.fit(train_ds,epochs=4,validation_data=validation_ds)

Epoch 1/4
625/625 ━━━━━━━━━━━━━━━━━━━━ 295s 473ms/step - accuracy: 0.9818 - loss: 0.0467 - val_accuracy: 0.9808 - val_loss: 0.0472
Epoch 2/4
625/625 ━━━━━━━━━━━━━━━━━━━━ 284s 412ms/step - accuracy: 0.9868 - loss: 0.0364 - val_accuracy: 0.9788 - val_loss: 0.0543
Epoch 3/4
625/625 ━━━━━━━━━━━━━━━━━━━━ 263s 414ms/step - accuracy: 0.9878 - loss: 0.0315 - val_accuracy: 0.9812 - val_loss: 0.0470
Epoch 4/4
625/625 ━━━━━━━━━━━━━━━━━━━━ 293s 463ms/step - accuracy: 0.9889 - loss: 0.0304 - val_accuracy: 0.9808 - val_loss: 0.0512


In [ ]:
#Stacking using meta model
vgg16_base=VGG16(input_tensor=Inputs,include_top=False)
resnet_base=ResNet50(input_tensor=Inputs,include_top=False)
densenet_base=DenseNet121(input_tensor=Inputs,include_top=False)
vgg16_out=Flatten()(vgg16_base.output)
resnet_out=Flatten()(resnet_base.output)
densenet_out=Flatten()(densenet_base.output)
vgg16_out=Dense(1,activation="sigmoid")(vgg16_out)
densenet_out=Dense(1,activation="sigmoid")(densenet_out)
resnet_out=Dense(1,activation="sigmoid")(resnet_out)
vgg_model=Model(inputs=Inputs,outputs=vgg16_out)
vgg_model.compile(optimizer="adam",loss="binary_crossentropy",metrics=["accuracy"])
resnet_model=Model(inputs=Inputs,outputs=resnet_out)
resnet_model.compile(optimizer="adam",loss="binary_crossentropy",metrics=["accuracy"])
densenet_model=Model(inputs=Inputs,outputs=densenet_out)
densenet_model.compile(optimizer="adam",loss="binary_crossentropy",metrics=["accuracy"])
for layer in vgg16_base.layers:
    layer.name = "vggbase_" + layer.name
for layer in resnet_base.layers:
    layer.name = "resnetbase_" + layer.name
for layer in densenet_base.layers:
    layer.name = "densenetbase_" + layer.name
for model in [vgg16_base,resnet_base,densenet_base]:
  model.trainable=False

In [ ]:
vgg_model.fit(train_ds,epochs=2,validation_data=validation_ds)
resnet_model.fit(train_ds,epochs=7,validation_data=validation_ds)
densenet_model.fit(train_ds,epochs=1,validation_data=validation_ds)

Epoch 1/2
625/625 ━━━━━━━━━━━━━━━━━━━━ 143s 228ms/step - accuracy: 0.9786 - loss: 0.0614 - val_accuracy: 0.9242 - val_loss: 0.1863
Epoch 2/2
625/625 ━━━━━━━━━━━━━━━━━━━━ 155s 248ms/step - accuracy: 0.9830 - loss: 0.0539 - val_accuracy: 0.9308 - val_loss: 0.1740
Epoch 1/7
625/625 ━━━━━━━━━━━━━━━━━━━━ 64s 103ms/step - accuracy: 0.7274 - loss: 0.5616 - val_accuracy: 0.7184 - val_loss: 0.5781
Epoch 2/7
625/625 ━━━━━━━━━━━━━━━━━━━━ 64s 102ms/step - accuracy: 0.7472 - loss: 0.5154 - val_accuracy: 0.7358 - val_loss: 0.5408
Epoch 3/7
625/625 ━━━━━━━━━━━━━━━━━━━━ 71s 114ms/step - accuracy: 0.7235 - loss: 0.5764 - val_accuracy: 0.6754 - val_loss: 0.6923
Epoch 4/7
625/625 ━━━━━━━━━━━━━━━━━━━━ 75s 103ms/step - accuracy: 0.7407 - loss: 0.5350 - val_accuracy: 0.7090 - val_loss: 0.6094
Epoch 5/7
625/625 ━━━━━━━━━━━━━━━━━━━━ 64s 102ms/step - accuracy: 0.7349 - loss: 0.5555 - val_accuracy: 0.7610 - val_loss: 0.5017
Epoch 6/7
625/625 ━━━━━━━━━━━━━━━━━━━━ 64s 102ms/step - accuracy: 0.7625 - loss: 0.4965 

In [ ]:
vgg_pred=vgg_model.predict(validation_ds)
resnet_pred=resnet_model.predict(validation_ds)
densenet_pred=densenet_model.predict(validation_ds)

157/157 ━━━━━━━━━━━━━━━━━━━━ 28s 178ms/step
157/157 ━━━━━━━━━━━━━━━━━━━━ 13s 85ms/step
157/157 ━━━━━━━━━━━━━━━━━━━━ 12s 74ms/step


In [ ]:
import numpy as np
y = np.array([y.numpy() for _, y in validation_ds.unbatch()])
y.shape

(5000,)

In [ ]:
x=np.concatenate((vgg_pred,resnet_pred,densenet_pred),axis=1)
x.shape

(5000, 3)

In [ ]:
#Meta model
import sklearn
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y)
from sklearn.linear_model import LogisticRegression
model=LogisticRegression()
model.fit(x_train,y_train)
y_pred=model.predict(x_test)
from sklearn.metrics import classification_report
report=classification_report(y_test,y_pred)

In [ ]:
print(report)

              precision    recall  f1-score   support

           0       0.51      0.54      0.52       625
           1       0.51      0.49      0.50       625

    accuracy                           0.51      1250
   macro avg       0.51      0.51      0.51      1250
weighted avg       0.51      0.51      0.51      1250



In [ ]:
#Instaed of using meta model
y_pred=[]
for i in x:
  if (i[0]>0.5 and i[1]>0.5)or(i[1]>0.5 and i[2]>0.5)or(i[2]>0.5 and i[0]>0.5):
    y_pred.append(1)
  else:
    y_pred.append(0)
print(classification_report(y,y_pred))


              precision    recall  f1-score   support

           0       0.49      0.45      0.47      2500
           1       0.49      0.53      0.51      2500

    accuracy                           0.49      5000
   macro avg       0.49      0.49      0.49      5000
weighted avg       0.49      0.49      0.49      5000



In [ ]:
#Instead of using resnet
x=np.concatenate((vgg_pred,densenet_pred),axis=1)
x.shape

(5000, 2)

In [ ]:
#Meta model
import sklearn
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y)
from sklearn.linear_model import LogisticRegression
model=LogisticRegression()
model.fit(x_train,y_train)
y_pred=model.predict(x_test)
from sklearn.metrics import classification_report
report=classification_report(y_test,y_pred)

In [ ]:
print(report)

              precision    recall  f1-score   support

           0       0.53      0.54      0.54       628
           1       0.53      0.51      0.52       622

    accuracy                           0.53      1250
   macro avg       0.53      0.53      0.53      1250
weighted avg       0.53      0.53      0.53      1250



In [ ]:
#Instaed of using meta model
y_pred=[]
for i in x:
  if (i[0]>0.5 and i[1]>0.5):
    y_pred.append(1)
  else:
    y_pred.append(0)
print(classification_report(y,y_pred))

              precision    recall  f1-score   support

           0       0.50      0.75      0.60      2500
           1       0.49      0.25      0.33      2500

    accuracy                           0.50      5000
   macro avg       0.50      0.50      0.46      5000
weighted avg       0.50      0.50      0.46      5000

